## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Strezhevoy,RU,60.7333,77.5889,12.20,85,90,15.66,light snow
1,1,Airai,TL,-8.9266,125.4092,66.85,100,100,1.54,moderate rain
2,2,Iqaluit,CA,63.7506,-68.5145,-4.00,77,90,2.30,light snow
3,3,Katsuura,JP,35.1333,140.3000,46.99,51,75,5.99,broken clouds
4,4,Carahue,CL,-38.7000,-73.1667,42.80,100,40,2.30,fog


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Airai,TL,-8.9266,125.4092,66.85,100,100,1.54,moderate rain
6,6,Lompoc,US,34.6391,-120.4579,63.00,71,0,7.29,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,78.58,73,0,11.90,clear sky
11,11,Ancud,CL,-41.8697,-73.8203,51.80,93,90,5.75,overcast clouds
12,12,Piacabucu,BR,-10.4056,-36.4344,75.54,83,0,8.61,clear sky
13,13,Mar Del Plata,AR,-38.0023,-57.5575,60.01,94,90,8.05,light rain
14,14,Vaini,TO,-21.2000,-175.2000,82.40,89,75,4.61,broken clouds
19,19,Pacific Grove,US,36.6177,-121.9166,51.80,100,90,7.20,fog
22,22,Altamira,BR,-3.2033,-52.2064,71.91,88,31,1.92,scattered clouds
23,23,Mataura,NZ,-46.1927,168.8643,54.88,78,100,4.99,moderate rain


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 1 to 719
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              393 non-null    int64  
 1   City                 393 non-null    object 
 2   Country              391 non-null    object 
 3   Lat                  393 non-null    float64
 4   Lng                  393 non-null    float64
 5   Max Temp             393 non-null    float64
 6   Humidity             393 non-null    int64  
 7   Cloudiness           393 non-null    int64  
 8   Wind Speed           393 non-null    float64
 9   Current Description  393 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 33.8+ KB


In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Airai,TL,66.85,moderate rain,-8.9266,125.4092,
6,Lompoc,US,63.00,clear sky,34.6391,-120.4579,
7,Rikitea,PF,78.58,clear sky,-23.1203,-134.9692,
11,Ancud,CL,51.80,overcast clouds,-41.8697,-73.8203,
12,Piacabucu,BR,75.54,clear sky,-10.4056,-36.4344,
13,Mar Del Plata,AR,60.01,light rain,-38.0023,-57.5575,
14,Vaini,TO,82.40,broken clouds,-21.2000,-175.2000,
19,Pacific Grove,US,51.80,fog,36.6177,-121.9166,
22,Altamira,BR,71.91,scattered clouds,-3.2033,-52.2064,
23,Mataura,NZ,54.88,moderate rain,-46.1927,168.8643,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found for... skipping")
        

Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping
Hotel not found for... skipping


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Airai,TL,66.85,moderate rain,-8.9266,125.4092,Juvinal Martin
6,Lompoc,US,63.00,clear sky,34.6391,-120.4579,Holiday Inn Express Lompoc
7,Rikitea,PF,78.58,clear sky,-23.1203,-134.9692,Pension Maro'i
11,Ancud,CL,51.80,overcast clouds,-41.8697,-73.8203,Hotel Arena Gruesa
12,Piacabucu,BR,75.54,clear sky,-10.4056,-36.4344,Piaçabuçu News
...,...,...,...,...,...,...,...
708,Champerico,GT,64.99,clear sky,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
711,Naze,JP,55.40,broken clouds,28.3667,129.4833,奄美ポートタワーホテル
714,Pasighat,IN,77.79,clear sky,28.0667,95.3333,Siang Guest House
717,Arya,PK,64.89,broken clouds,33.6000,73.0667,Jungle Barracks


In [33]:
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 1 to 719
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 391 non-null    object 
 1   Country              391 non-null    object 
 2   Max Temp             391 non-null    float64
 3   Current Description  391 non-null    object 
 4   Lat                  391 non-null    float64
 5   Lng                  391 non-null    float64
 6   Hotel Name           391 non-null    object 
dtypes: float64(3), object(4)
memory usage: 24.4+ KB


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))